### ORB OpenCV

In [1]:
import numpy as np
import pickle
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def pickle_keypoints(keypoints, descriptors):
    i = 0
    temp_array = []
    for point in keypoints:
        temp = (point.pt, point.size, point.angle, point.response, point.octave, point.class_id, descriptors[i])     
        ++i
        temp_array.append(temp)
    return temp_array

In [3]:
def unpickle_keypoints(array):
    keypoints = []
    descriptors = []
    for point in array:
        temp_feature = cv2.KeyPoint(x=point[0][0],y=point[0][1],_size=point[1], _angle=point[2], _response=point[3], _octave=point[4], _class_id=point[5])
        temp_descriptor = point[6]
        keypoints.append(temp_feature)
        descriptors.append(temp_descriptor)
    return keypoints, np.array(descriptors)

In [4]:
cap = cv2.VideoCapture(0)

#saved_features = []
#saved_des = []

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # Initiate STAR detector
    orb = cv2.ORB_create(400)
    
    # find the keypoints with ORB
    kp = orb.detect(gray,None)
    
    # compute the descriptors with ORB
    kp, des = orb.compute(gray, kp)
    
    # draw only keypoints location,not size and orientation
    img = cv2.drawKeypoints(gray,kp,None, flags=0)
    
    cv2.imshow('dst',img)
    
    if cv2.waitKey(33) & 0xFF == ord('s'):
        saved_features = pickle_keypoints(kp, des)
        pickle.dump(saved_features, open("saved_features.p", "wb"))
        print("ORB Features Saved...")
        #saved_features.append(kp)
        #saved_des.append(des)
        break
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

ORB Features Saved...


In [5]:
saved_features

[((321.0, 42.0),
  31.0,
  84.5276107788086,
  0.0011959379771724343,
  0,
  -1,
  array([246, 120, 239, 116,  75, 108, 157,   4, 169, 246, 164, 185,  62,
         189,  69,  27, 165, 245, 253, 131, 105,  71, 178,  64, 211, 115,
          93, 187,  69, 113,  70, 123], dtype=uint8)),
 ((520.0, 111.0),
  31.0,
  9.20598316192627,
  0.0002642024483066052,
  0,
  -1,
  array([246, 120, 239, 116,  75, 108, 157,   4, 169, 246, 164, 185,  62,
         189,  69,  27, 165, 245, 253, 131, 105,  71, 178,  64, 211, 115,
          93, 187,  69, 113,  70, 123], dtype=uint8)),
 ((162.0, 336.0),
  31.0,
  7.52578067779541,
  0.0002828882134053856,
  0,
  -1,
  array([246, 120, 239, 116,  75, 108, 157,   4, 169, 246, 164, 185,  62,
         189,  69,  27, 165, 245, 253, 131, 105,  71, 178,  64, 211, 115,
          93, 187,  69, 113,  70, 123], dtype=uint8)),
 ((387.0, 44.0),
  31.0,
  267.9729309082031,
  0.00023299692838918418,
  0,
  -1,
  array([246, 120, 239, 116,  75, 108, 157,   4, 169, 246, 164,

### TEST With New Image

In [6]:
# cap = cv2.VideoCapture(0)

# good_match_points = []
# saved_des2 = []

# #Retrieve Saved Keypoint Features
# keypoints_database = pickle.load(open( "saved_features.p", "rb" ))
# kp1, des1 = unpickle_keypoints(keypoints_database)

# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
#     # Initiate STAR detector
#     orb = cv2.ORB_create(400)
    
#     # find the keypoints with ORB
#     kp2 = orb.detect(gray,None)
    
#     # compute the descriptors with ORBq
#     kp2, des2 = orb.compute(gray, kp2)
    
#     # draw only keypoints location,not size and orientation
#     img1 = cv2.drawKeypoints(gray,kp2,None, flags=0)
    
#     cv2.imshow('dst',img1)
    
#     # create BFMatcher object
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
#     # Match Saved Data Points with New Points
#     if cv2.waitKey(33) & 0xFF == ord('c'):
#         print("Checking for Match...\n")
#         # Match descriptors.
#         matches = bf.match(des1,des2)
#         # Sort them in the order of their distance.
#         matches = sorted(matches, key = lambda x:x.distance)
#         break
        
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

### TEST 2

In [7]:
def tune_ratio(matches, ratio=0.75):
    print("Tuning Ratio....")
    good = []
    for m,n in matches:
        if m.distance < ratio * n.distance:
            good.append([m])
    if (len(good) > 0):
        print("SCENE MATCHED....")
        return good
    else:
        if (ratio > 0.30):
            ratio = ratio - 0.05
            tune_ratio(matches, ratio)
        else:
            print("Unable to Tune, Exiting....")
            return good
    return good

In [8]:
cap = cv2.VideoCapture(0)

good_match_points = []
saved_des2 = []

#Retrieve Saved Keypoint Features
keypoints_database = pickle.load(open( "saved_features.p", "rb" ))
kp1, des1 = unpickle_keypoints(keypoints_database)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # Initiate STAR detector
    orb = cv2.ORB_create(400)
    
    # find the keypoints with ORB
    kp2 = orb.detect(gray,None)
    
    # compute the descriptors with ORBq
    kp2, des2 = orb.compute(gray, kp2)
    
    # draw only keypoints location,not size and orientation
    img1 = cv2.drawKeypoints(gray,kp2,None, flags=0)
    
    cv2.imshow('dst',img1)
    
    # Matcher
    FLANN_INDEX_LSH = 6
    index_params= dict(algorithm = FLANN_INDEX_LSH,
                   table_number = 6, # 12
                   key_size = 12,     # 20
                   multi_probe_level = 1) #2
    search_params = dict(checks=60) 
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    # Match Saved Data Points with New Points
    if cv2.waitKey(33) & 0xFF == ord('c'):
        print("Checking for Match...\n")
        # Match descriptors.
        matches = flann.knnMatch(des1,des2,k=2)
        # Need to draw only good matches, so create a mask
        #matchesMask = [[0,0] for i in range(len(matches))]
        
        # Apply ratio test
        good = []
        for m,n in matches:
            if m.distance < 0.75 * n.distance:
                good.append([m])       

        print("Number of Good Matches: ", len(good))
        if (len(good) > 0):
            print("SCENE MATCHED....")
        else:
            print("SCENE NOT MATCHED....")
            good = tune_ratio(matches, ratio=0.75)
        break
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Checking for Match...

Number of Good Matches:  0
SCENE NOT MATCHED....
Tuning Ratio....
Tuning Ratio....
Tuning Ratio....
Tuning Ratio....
Tuning Ratio....
Tuning Ratio....
Tuning Ratio....
Tuning Ratio....
Tuning Ratio....
Tuning Ratio....
Unable to Tune....
